[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/06-agent-executor.ipynb)

#### LangChain Essentials Course

# LangChain Agent Executor

In this chapter, we will continue from the [introduction to agents](https://aurelio.ai/learn/langchain-agents-intro) and dive deeper into agents. Learning how to build our custom agent execution loop for v0.3 of LangChain.

In [3]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3 \
  langchain-groq==0.2.0 \
  langchain-community==0.3.16 \
  langsmith==0.3.4

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [4]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "default"

Enter LangSmith API Key: ··········


---

## What is the Agent Executor?

When we talk about agents, a significant part of an "agent" is simple code logic,
iteratively rerunning LLM calls and processing their output. The exact logic varies
significantly, but one well-known example is the **ReAct** agent.

![ReAct process](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-00.png&w=640&q=75)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to
incorporate chain-of-thought and tool-use into their execution. During the _reasoning_
step, the LLM generates the steps to take to answer the query. Next, the LLM generates
the _action_ input, which our code logic parses into a tool call.

![Agentic graph of ReAct](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-01.png&w=640&q=75)

Following our action step, we get an observation from the tool call. Then, we feed the
observation back into the agent executor logic for a final answer or further reasoning
and action steps.

The agent and agent executor we will be building will follow this pattern.

## Tools

As with the previous chapter, we will define a few tools to use within our agent.

In [4]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [6]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x799db1ef80e0>)

We can see the tool name, description, and arg schema:

In [7]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [8]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

The `args_schema` is a pydantic model that is transformed into the JSON schema above and passed into our LLM, it is this that defines _how_ the tool is used for the LLM. We can see this with other tools:

In [9]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [10]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [11]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent writes its notes as it works through multiple internal thought and tool-use steps to produce a final output for the user. This scratchpad is a list of messages with alternating roles of `ai` (for the tool call) and `tool` (for the tool execution output). Both message types require a `tool_call_id` field which is used to link the respective AI and tool messages - this can be required when we many tool calls happening in parallel.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

Next, we must define our LLM, we will use the `gpt-4o-mini` model with a `temperature` of `0.0`.

In [ ]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
)

Enter your OpenAI API key: ··········


In [1]:
# Install required packages
!pip install langchain-groq==0.2.0 langchain-core==0.3.0 pydantic==2.7.0

# Import modules
from langchain_groq import ChatGroq
from langchain_core.caches import BaseCache
import os

# Rebuild the model to resolve Pydantic issues
ChatGroq.model_rebuild()

# Set your Groq API key (replace with your actual key)
os.environ["GROQ_API_KEY"] = "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"

# Instantiate ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

To add tools to our LLM, we will use the `bind_tools` method within the LCEL constructor, which will take our tools and add them to the LLM. We'll also include the `tool_choice="any"` argument to `bind_tools`, which tells the LLM that it _MUST_ use a tool, ie it cannot provide a final answer directly (in therefore not using a tool):

In [8]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

We invoke the agent with the `invoke` method, passing in the input and chat history.

In [9]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bf18', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 488, 'total_tokens': 506, 'completion_time': 0.024, 'prompt_time': 0.028991773, 'queue_time': 0.393067194, 'total_time': 0.052991773}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f7bd09b454', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3bd7c846-fa4e-4307-9c7d-7583200a85cc-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_bf18', 'type': 'tool_call'}], usage_metadata={'input_tokens': 488, 'output_tokens': 18, 'total_tokens': 506})

Because we set `tool_choice="any"` to force the tool output, the usual `content` field will be empty as that field is used for natural language output, ie the _final answer_ of the LLM. To find our tool output, we need to look at the `tool_calls` field:

In [10]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_bf18',
  'type': 'tool_call'}]

From here, we have the tool `name` that our LLM wants to use and the `args` that it
wants to pass to that tool. We can see that the tool `add` is being used with the
arguments `x=10` and `y=10`. The `agent.invoke` method has _not_ executed the tool
function; we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool `name` to the tool function.

2. Execute the tool function with the generated `args`.

In [11]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

Now execute to get our answer:

In [12]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

20

That is our answer and tool execution logic. We feed this back into our LLM via the
`agent_scratchpad` placeholder.

In [18]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='20', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 522, 'total_tokens': 524, 'completion_time': 0.002666667, 'prompt_time': 0.028274116, 'queue_time': 0.19814840900000003, 'total_time': 0.030940783}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run-76ff5bab-c362-4626-af3d-3101e71e7243-0', usage_metadata={'input_tokens': 522, 'output_tokens': 2, 'total_tokens': 524})

Despite having the answer in our `agent_scratchpad`, the LLM still tries to use the tool
_again_. This behaviour happens because we bonded the tools to the LLM with
`tool_choice="any"`. When we set `tool_choice` to `"any"` or `"required"`, we tell the
LLM that it _MUST_ use a tool, i.e., it cannot provide a final answer.

There's two options to fix this:

1. Set `tool_choice="auto"` to tell the LLM that it can choose to use a tool or provide
a final answer.

2. Create a `final_answer` tool - we'll explain this shortly.

First, let's try option **1**:

In [19]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

We'll start from the start again, so `agent_scratchpad` is empty:

In [20]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3p7h', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 488, 'total_tokens': 506, 'completion_time': 0.024, 'prompt_time': 0.023051878, 'queue_time': 0.198621258, 'total_time': 0.047051878}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9a5cea6a-3ee9-4c6f-ae07-99fb1deed298-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_3p7h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 488, 'output_tokens': 18, 'total_tokens': 506})

Now we execute the tool and pass it's output into the `agent_scratchpad` placeholder:

In [21]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='20', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 522, 'total_tokens': 524, 'completion_time': 0.002666667, 'prompt_time': 0.037712629, 'queue_time': 0.197422098, 'total_time': 0.040379296}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff6e1f23-7c79-4f13-b553-4acfad7a2f8a-0', usage_metadata={'input_tokens': 522, 'output_tokens': 2, 'total_tokens': 524})

We now have the final answer in the `content` field! This method is perfectly
functional; however, we recommend option **2** as it provides more control over the
agent's output.

There are several reasons that option **2** can provide more control, those are:

* It removes the possibility of an agent using the direct `content` field when it is not
appropriate; for example, some LLMs (particularly smaller ones) may try to use the
`content` field when using a tool.

* We can enforce a specific structured output in our answers. Structured outputs are
handy when we require particular fields for downstream code or multi-part answers. For
example, a RAG agent may return a natural language answer and a list of sources used to
generate that answer.

To implement option **2**, we must create a `final_answer` tool. We will add a
`tools_used` field to give our output some structure—in a real-world use case, we
probably wouldn't want to generate this field, but it's useful for our example here.

In [22]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

Our `final_answer` tool _doesn't_ necessarily need to do anything; in this example,
we're using it purely to structure our final response. We can now add this tool to our
agent:

In [24]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")  # we're forcing tool use again
)

Now we invoke:

In [25]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_2mbt',
  'type': 'tool_call'},
 {'name': 'final_answer',
  'args': {'answer': 'The result of 10 + 10 is 20.', 'tools_used': ['add']},
  'id': 'call_1z5w',
  'type': 'tool_call'}]

We execute the tool and provide it's output to the agent again:

In [30]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='The result of 10 + 10 is 20.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 715, 'total_tokens': 728, 'completion_time': 0.017333333, 'prompt_time': 0.078634566, 'queue_time': 0.216807596, 'total_time': 0.095967899}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f7bd09b454', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e45a5ef-285e-4144-b03d-6a413d177426-0', usage_metadata={'input_tokens': 715, 'output_tokens': 13, 'total_tokens': 728})

We see that `content` remains empty because we force tool use. But we now have the
`final_answer` tool, which the agent executor passes via the `tool_calls` field:

In [31]:
out.tool_calls

[]

Because we see the `final_answer` tool here, we don't pass this back into our agent, and
instead, this tells us to stop execution and pass the `args` output onto our downstream
process or user directly:

In [45]:
out.tool_calls[0]["args"]

IndexError: list index out of range

### Building a Custom Agent Execution Loop

We've worked through each step of our agent code, but it doesn't run without us running
every step. We must write a class to handle all the logic we just worked through.

In [46]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import json


class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="auto")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            # add a print so we can see intermediate steps
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history
        final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        # return the final answer in dict form
        return json.dumps(tool_out)

Now initialize the agent executor:

In [42]:
agent_executor = CustomAgentExecutor()

And test the `invoke` method:

In [49]:
agent_executor.invoke(input="What is 10*5 + 130")

0: multiply({'x': 10, 'y': 5})


IndexError: list index out of range

We then get our answer and the tools that were used — all through our custom agent
executor.